# Programming Exercise 4, Hidden Markov Model

The goal of this task is to learn about the how to implement the Hidden Markov Model, especially the Viterbi algorithm.
Then use the model to do smoothing and make predictions

The coronavirus is spreading in Munich and threatening our health. In this exercise, we would like to model the spread of the virus using an HMM model. For this purpose, we set 7 states of pandemic severity (1 for least severe, 7 for most severe). Then, we collect the corona data from April 2020 to December 2021 (21 months in total) and compute the number of new cases and deaths for each month as our observation. Based on this, our task is to find the most likely severity sequence using the Viterbi algorithm, do smoothing, and make a prediction about the severity of future pandemics.

Your tasks is to complete the missing code. Make sure that all the functions follow the provided interfaces of the functions, i.e. the output of the function exactly matches the description in the docstring.
Adding or modifying code outside of the following comment blocks is not required:

```
##########################################################
# YOUR CODE HERE
.....
##########################################################
```

### Learning Outcomes:

* implement prior matrix, transition model, sensor model
* implement viterbi algorithm

### Import

For testing and grading, we want to state that you are not allowed to import 
any other libraries and should not change the structure of the provided functions 
(i.a. the arguments and the name of the functions).

In [1]:
import branca.colormap as cmap
import folium
from folium.plugins import TimeSliderChoropleth
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import multivariate_normal

### Corona data visualization (Nothing to implement here)

Before we start to model the corona situation, we would like to visualize the data first to have a basic understanding of it. Feel free to change some parameters to achieve different visualization settings. Have fun!

In [2]:
# load the corona data
corona_df = pd.read_csv('coronadata_Munich.csv')
munich_map_df = gpd.read_file('map/vg2500_krs.shp')

In [3]:
# check what munich_map_df looks like
munich_map_df

In [4]:
# We only need the GEN and geometry columns
munich_map_df = munich_map_df[['GEN', 'geometry']]

# Only keep Munich and LK Munich
munich_map_df = munich_map_df.loc[[224, 239]]
munich_map_df['GEN'] = ['Munich','LK Munich']

In [5]:
# check what corona_df looks like
corona_df

In [6]:
# Both dataframes have the column "GEN", we merge them using this column
joined_df = corona_df.merge(munich_map_df, on='GEN')

# The ObservationDate is given in date and time, we convert it to unix time in nanoseconds
joined_df['date_sec'] = pd.to_datetime(joined_df['ObservationDate']).astype(np.int64) / 10**9
joined_df['date_sec'] = joined_df['date_sec'].astype(int).astype(str)

# Delete the ObservationDate column as we do not need it anymore
joined_df = joined_df.drop('ObservationDate', axis = 1)

# Check the final dataframe before we do the visualization
joined_df

C:\Users\10122\AppData\Local\Temp/ipykernel_7336/1281155843.py:5: FutureWarning: casting datetime64[ns, UTC] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  joined_df['date_sec'] = pd.to_datetime(joined_df['ObservationDate']).astype(np.int64) / 10**9


In [7]:
## Here, we visualize the daily confirmed case
# Add the color to each row
max_colour = max(joined_df['Confirmed'])
min_colour = min(joined_df['Confirmed'])
colour_map = cmap.linear.YlOrRd_09.scale(min_colour, max_colour)
joined_df['colour'] = joined_df['Confirmed'].map(colour_map)

# uncomment the following code to visualize daily death cases
#max_colour = max(joined_df['Deaths'])
#min_colour = min(joined_df['Deaths'])
#colour_map = cmap.linear.YlOrRd_09.scale(min_colour, max_colour)
#joined_df['colour'] = joined_df['Deaths'].map(colour_map)

# create an inner dictionary for the visualization
geo_list = joined_df['GEN'].unique().tolist()
geo_idx = range(len(geo_list))

style_dict = {}
for i in geo_idx:
    geo = geo_list[i]
    result = joined_df[joined_df['GEN'] == geo]
    inner_dict = {}
    for _, r in result.iterrows():
        inner_dict[r['date_sec']] = {'color': r['colour'], 'opacity': 0.7}
    style_dict[str(i)] = inner_dict

# create a geo_gdf for the visualization
geo_df = joined_df[['geometry']]
geo_gdf = gpd.GeoDataFrame(geo_df)
geo_gdf = geo_gdf.drop_duplicates().reset_index()

#You might need to change the value of min_zoom depending on your platform
slider_map = folium.Map(location=[48.08,11.61], min_zoom=2, max_bounds=True)

_ = TimeSliderChoropleth(data=geo_gdf.to_json(),styledict=style_dict).add_to(slider_map)
_ = colour_map.add_to(slider_map)
colour_map.caption = "confirmed cases in the past 7 days per 100,000 people"

## uncomment for the death cases
#colour_map.caption = "death cases in the past 7 days per 5,000,000 people"

## uncomment the following code to save the figure
#slider_map.save(outfile='daily_confirmed_case.html')
#slider_map.save(outfile='daily_death_case.html')

In [8]:
# final visualization, drag the bar to see the statistics on different days
slider_map

### Build the HMM model

In our HMM model, there are two observations: confirmed cases per 100,000 people and death cases per 5,000,000 people. Unlike the umbrella case in our textbook, our data is continuous, which means we need a continuous HMM model. But don't worry! The transition from a discrete HMM model to a continuous one is easy. In this task, you are only supposed to implement a discrete model.

Let's define a class named HMM that is able to execute filtering, smoothing, prediction and the most likely path estimation based on discrete observation data. You will implement the aforementioned functions step by step within the **HMM** class. Vectorized operation given in the textbook will be very useful for during implementation.

**Note: During implementation, we refer the state with their associated indices.** For example, as defined in the textbook, the ij-th entry of transition matrix denotes the transformation probability from state j to state i.

In [9]:
def dict_to_array(state_dict):
    state_array = [state_dict[t] for t in sorted(state_dict.keys())]        
    return np.array(state_array) 

class HMM():
    def __init__(self, 
                 observation_mu,
                 observation_sigma,
                 transition_matrix, 
                 initial_probability):
        assert len(transition_matrix) == len(observation_mu) == len(observation_sigma)
        self.trans_mat = transition_matrix
        self.obs_mat = [multivariate_normal(observation_mu[i], observation_sigma[i]) 
                        for i in range(len(observation_mu))]
        self.init_prob = initial_probability
        self.N_state = len(self.trans_mat)        
        
    def compute_observation_matrix(self, observation):
        """ Compute the observation matrix for vectorized operation.
       
        Args:
            observation: observation in some timestamp

        Return: 
            O_matrix: observation matrix

        """
        prob_density = [self.obs_mat[i].pdf(observation) for i in range(self.N_state)]
        O_matrix = np.diag(prob_density)
        return O_matrix


    def forward_onestep(self, f, observation):
        """ Compute one forward step for filtering.
            N stands for number of hidden states.
            Hint: Use the O_matrix we provided for one step forward oepration.

        Args:
            f: numpy array with shape [N, ], vector of f_{1:t} depicting probability of state given previous observation sequence
            observation: observed state in timestamp t+1
            

        Return: 
            f_onestep: numpy array with shape [N, ], updated vector of f_{1:t+1}

        """
        # Acquire the row vector and transform it to diagonal matrix
        O_matrix = self.compute_observation_matrix(observation)
        f_onestep = None
        #######################################
        # YOUR CODE HERE
        
        #######################################
        return f_onestep
        
        
    def backward_onestep(self, b, observation):
        """ Compute one backward step for smoothing 
            N stands for number of hidden states. 
            Hint: Use the O_matrix we provided for one step backward.

        Args:
            b: numpy array with shape [N, ], vector of b_{k+2:T} depicting probability of observation sequence given state
            observation: observed state in timestamp k+1
            
        Return: 
            b_onestep: numpy array with shape [N, ], updated vector of b_{k+1:T}

        """
        # Acquire the row vector and transform it to diagonal matrix
        O_matrix = self.compute_observation_matrix(observation)
        b_onestep = None
        #######################################
        # YOUR CODE HERE
        
        #######################################
        return b_onestep
        
        
    def forward_backward(self, observation_sequence):
        """ forward-backward algorithm for smoothing
            In this function, you will finalize the forward-backward algorithm based on your previous implementaions
            of function “forward_onestep” and "backward_onestep".
            
            Remember to normalize the result smoothed probabilty to ensure the probability sum to be 1! 
            Note: We provide normalization function as a reference, you can also implement your own :)
       
            T stands for sequence lenghth and N stands for number of hidden states.

        Args:
            observation_sequence: observed sequence in a given period with length T
            

        Return: 
            smoothed_prob: numpy array with shape [T, N], state probability in this period after smoothing.

        """
        forward_prob = {}
        smoothed_prob = {}
        backward_prob = {}
        #######################################
        # YOUR CODE HERE
        
        #######################################
        return dict_to_array(smoothed_prob)

        
    def predict(self, observation_sequence, k):
        """ Predict state probability for future timestamp, remember to filter the given observation sequence at first.
            
            Remember to normalize the result forward probabilty to ensure the probability sum to be 1! 
            Note: We provide normalization function as a reference, you can also implement your own :)
            
            N stands for number of hidden states.

        Args:
            observation_sequence: observed sequence in a given period with length T
            k: the number of steps of timestamp T
            
        Return: 
            p: numpy array with shape [N, ], vector of state probability after k steps of timestamp T in the future
             
        """
        #######################################
        # YOUR CODE HERE
        
        #######################################
        return p
        

    def viterbi(self, observation_sequence):
        """ Compute the most likely state trajectory given a observation sequence.
            T stands for sequence lenghth.

        Args:
            observation_sequence: observed sequence in a given period with length T
            
        Return: 
            trajectory: numpy array with shape [T, ], trajectory of the most likely state in the given period.
            Note that the output trajectory is in the form of the corresponding indices of the states. For example,
            the output with numpy array [1, 7, 5] denotes given a sequence with lenghth T=3, the most likely state 
            trasnforms from state 1 to state 7 and finally to state 5.
        """
        max_prob = {}
        most_likely_state = {}
        trajectory = {}
        #######################################
        # YOUR CODE HERE
        
        #######################################
        return dict_to_array(trajectory)

    @staticmethod
    def normalize(logit, axis=0):
        prob = logit / np.sum(logit, axis=axis)
        return prob 

### Test Your Implementations

Let's run your implemented functions with some dummy data.

**Please DON'T change the pre-defined parameters below!**

In [10]:
# Pre-defined model parameters for HMM testing
# Input transition matrix, NxN matrix in our scenario (N states)
T = np.array([[0.2, 0.6],
              [0.8, 0.4]])

# Input mean of gaussian variable here, Nx2 matrix in our scenario,
# Since each state has its own corresponding gaussian distribution (N states), and the dimension of the observation is H.
# For each H-dimensional gaussian distribution, its corresponding mean (mu) vector shall has H dimension.
# So we input 7xH matrix as *mus* in our scenario.
mus = np.array([[0.8, 0.7], 
                [0.3, 0.2]])

# Input mean of gaussian variable here, NxHxH matrix in our scenario,
# Since each state has its own corresponding gaussian distribution (N states), and the dimension of the observation is H.
# For each H-dimensional gaussian distribution, its corresponding covariance matrix (sigma) shall be HxH.
# So we input NxHxH matrix as *sigmas* in our scenario.
sigmas = np.zeros((2, 2, 2))
sigmas[0] = np.eye(2)
sigmas[1] = np.eye(2)

# Input initialized state distribution here, a vector with N dimension. (N states)
init_prob = np.array([0.3, 0.7])

In [11]:
from utils import test_hmm

# Initialize your implemented HMM model!
hmm = HMM(mus, sigmas, T, init_prob) 

# Test each implemenation module :)
test_hmm('forward_backward', hmm)    
test_hmm('prediction', hmm)    
test_hmm('viterbi', hmm)   

### Apply HMM for COVID Severity Estimation

In this section, your final task is to use the use HMM model for COVID severity estimation as a guide for examination decision-makers. 

You may refer to this [paper](https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=9543670) to better understand the how we can leverage HMM to model the spread progress of COVID given some observation data such as daily and cumulative infections and daily deaths.

In this task, we define the following components of HMM as follows:

```
Hidden State: Severity of the COVID, there are 7 discrete levels in total and the transition diagram is shown in below.
Observation: A 7 * 2 matrix (mus) and a 7 * 2 * 2 matrix (sigmas)
Transformation Model: A 7 * 7 matrix that depicts the internal relationship between each state.
Sensor Model: A 2-dimensional gaussian distribution based on given state.
```

It can be a little bit tricky since in this scenario the sensor model is modeled as a continuous probability density distribution instead of a discrete matrix as discussed during lectures. While don't be panic since we have  provided you a helper function to extract the diagonal observation matrix (O_matrix), and then the algorithm for filtering, smoothing and viterbi will be just the same as the discrete case we dealt with during class ;) 

We will provide pre-estimated HMM parameters for you such as transformation matrix and preprocessed data, let's now take a look on how we use HMM to model COVID :)

### Transitional Matrix

We define the transitional matrix as the following figure shows:
![title](img/transitional_matrix.png)

In [12]:
#######################################
# T is a 7 * 7 matrix, T(ij) means the probability from state i to state j

# YOUR CODE HERE

#######################################

### Observation Matrix

We define the observational matrix as the following figure shows:
<img src='img/observational_matrix.png'  width='35%'  height='35%'>

In [13]:
#######################################
# mus is a 7 * 2 matrix, which indicates the input mean of gaussian variable
# sigmas is a 7 * 2 * 2 matrix, which indicates the input covariance of gaussian variable. Each state has a 2 * 2 covariance matrix

# YOUR CODE HERE

#######################################

### Initialized State Distribution (Nothing to do here)

In [14]:
# init_prob is a 7-dimensional vector
init_prob = np.array([0.414, 0, 0.401, 0.137, 0, 0, 0.047])

### Obsevation Sequence

Since we take into account 21 months in total, the sequence is a 21 * 2 matrix which contains both confirmed cases and death cases.

**The sequence can be found in sequence.csv**

In [15]:
#######################################
# obsevation sequence is a 21 * 2 matrix containing both confirmed cases and death cases, here, we define 2 sequences, one for Munich,
# one for LK_Munich

# YOUR CODE HERE

#######################################

### Testing (Nothing to change here)

In [16]:
# Test Forward-backward algorithm
hmm = HMM(mus, sigmas, T, init_prob)
smoothed_prob_Munich = hmm.forward_backward(obs_seq_Munich)
smoothed_prob_LK_Munich = hmm.forward_backward(obs_seq_LK_Munich)
print('smoothed_prob_Munich: \n', smoothed_prob_Munich, '\n')
print('smoothed_prob_LK_Munich: \n', smoothed_prob_LK_Munich, '\n')

# Test Prediction
predicted_prob_Munich = hmm.predict(obs_seq_Munich, 10)
predicted_prob_LK_Munich = hmm.predict(obs_seq_LK_Munich, 10)
print('prediction_Munich: \n', predicted_prob_Munich, '\n')
print('prediction_LK_Munich: \n', predicted_prob_LK_Munich, '\n')


# Test viterbi algorithm
trajectory_Munich = hmm.viterbi(obs_seq_Munich)
trajectory_LK_Munich = hmm.viterbi(obs_seq_LK_Munich)
print('trajectory_Munich: \n',trajectory_Munich, '\n')
print('trajectory_LK_Munich: \n',trajectory_LK_Munich, '\n')

**Nothing to change here**

Now that we have the trajectory, we can visualize the result using matplotlib or dataframe(pandas).

In [17]:
#visualize the result in plt

months = ['Apr 2020', 'May 2020', 'Jun 2020', 'Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020','Nov 2020', \
          'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021','Jul 2021', \
          'Aug 2021', 'Sep 2021', 'Oct 2021', 'Nov 2021', 'Dec 2021']

plt.figure(figsize=(20,20))

#Munich
plt.subplot(2, 1, 1)
fig_Munich = plt.step(months, trajectory_Munich, color="#8dd3c7", where="pre", lw=2)
plt.ylim(1, 7)
plt.title('Severity of COVID-19 in Munich')

#LK_Munich
plt.subplot(2, 1, 2)
fig_LK_Munich = plt.step(months, trajectory_LK_Munich, color="#8dd3c7", where="pre", lw=2)
plt.ylim(1, 7)
plt.title('Severity of COVID-19 in LK Munich')

In [18]:
#visualize the result in DataFrame
data = [months, list(trajectory_Munich), list(trajectory_LK_Munich)]
df = pd.DataFrame(np.transpose(data), columns=['Months', 'Munich','LK_Munich'])

df